<a href="https://colab.research.google.com/github/MRazin172/Reinforcement-Learning/blob/main/2348534_RL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

lets assume,

it is an environment with a finite number of states and actions.

and has defined rewards and transition probabilities.

In [19]:
import numpy as np


In [20]:
import numpy as np

class MDP:
    def __init__(self, states, actions, rewards, transition_probs, gamma=0.9):
        self.states = states
        self.actions = actions
        self.rewards = rewards
        self.transition_probs = transition_probs
        self.gamma = gamma
        self.values = np.zeros(len(states))
        self.policy = np.random.choice(actions, len(states))

    def policy_evaluation(self, threshold=1e-6):
        while True:
            delta = 0
            for state in self.states:
                v = self.values[state]
                action = self.policy[state]
                self.values[state] = sum(
                    self.transition_probs.get((state, action, next_state), 0) *
                    (self.rewards.get((state, action), 0) + self.gamma * self.values[next_state])
                    for next_state in self.states
                )
                delta = max(delta, abs(v - self.values[state]))
            if delta < threshold:
                break

    def policy_improvement(self):
        policy_stable = True
        for state in self.states:
            old_action = self.policy[state]
            action_values = []
            for action in self.actions:
                action_value = sum(
                    self.transition_probs.get((state, action, next_state), 0) *
                    (self.rewards.get((state, action), 0) + self.gamma * self.values[next_state])
                    for next_state in self.states
                )
                action_values.append(action_value)
            best_action = self.actions[np.argmax(action_values)]
            self.policy[state] = best_action
            if best_action != old_action:
                policy_stable = False
        return policy_stable

    def policy_iteration(self):
        iteration = 0
        while True:
            self.policy_evaluation()
            policy_stable = self.policy_improvement()
            iteration += 1
            if policy_stable:
                print(f"Optimal policy found after {iteration} iterations.")
                break
        return self.policy, self.values





The MDP class initializes the Markov Decision Process.

It accepts states, actions, rewards, transition_probs, and gamma (discount factor) as inputs.

self.values represents the estimated long-term values (returns) for each state, initialized to zero.

self.policy is initialized randomly, choosing a random action for each state.

The initialization creates a setup for tracking the current policy and estimated values for each state. Random initialization encourages exploration from various actions.



This method performs policy evaluation by iterating over each state and updating its value based on the current policy.

For each state, it calculates the expected return (self.values[state]) as the sum of potential rewards and future values from following the current policy.

It continues updating values until delta, the change in values from the previous iteration, is smaller than the threshold.

Observation: This iterative approach (value iteration) calculates the expected long-term reward for each state-action pair, given the current policy. When delta is small, it indicates convergence, meaning state values are stable under the policy.



This method improves the policy by selecting the action with the highest expected return for each state, based on current state values.

best_action is the action that maximizes the expected return, determined by evaluating all possible actions.

If best_action differs from old_action, policy_stable is set to False, indicating further improvements are possible.

Observation: This method attempts to optimize the policy by making greedy choices based on updated values. If no actions change, the policy is stable, signaling that the optimal policy has been found.

This method combines policy evaluation and policy improvement iteratively.

After each evaluation, it calls policy_improvement() to refine the policy.

If policy_stable is True, it indicates that further policy improvement won’t yield better returns, so the algorithm stops.

Observation: Policy iteration is an iterative approach that guarantees convergence to the optimal policy under certain conditions. It alternates between refining value estimates (policy evaluation) and optimizing actions (policy improvement).



In [21]:
states = [0, 1, 2, 3]  # Four example states
actions = [0, 1]       # Two possible actions
rewards = {
    (0, 0): 1, (0, 1): 0,
    (1, 0): 0, (1, 1): 2,
    (2, 0): 0, (2, 1): 3,
    (3, 0): 1, (3, 1): 0
}
transition_probs = {
    (0, 0, 0): 0.5, (0, 0, 1): 0.5,
    (0, 1, 2): 1.0,
    (1, 0, 2): 1.0,
    (1, 1, 3): 1.0,
    (2, 0, 3): 1.0,
    (2, 1, 0): 1.0,
    (3, 0, 0): 1.0,
    (3, 1, 1): 1.0
}

In [22]:
mdp = MDP(states, actions, rewards, transition_probs)
optimal_policy, optimal_values = mdp.policy_iteration()
print("Optimal Policy:", optimal_policy)
print("Optimal State Values:", optimal_values)


Optimal policy found after 4 iterations.
Optimal Policy: [1 1 1 0]
Optimal State Values: [14.21052352 14.41052352 15.78947117 13.78947117]



1. **Optimal Policy**: `[1 1 1 0]`
   - This result suggests that, for states 0, 1, and 2, action 1 is optimal, while for state 3, action 0 is optimal.
   - The actions chosen in the optimal policy maximize the expected cumulative rewards based on the given transition probabilities and rewards.
   
2. **Optimal State Values**: `[14.21, 14.41, 15.79, 13.79]`
   - These values represent the expected cumulative reward for each state under the optimal policy.
   - State 2 has the highest state value (15.79), which aligns with it being prioritized in action selection under the policy (since actions here should yield higher returns in the long run).
   - The close values between states 0, 1, and 2 suggest the policy and values are converging and that state 3 has a slightly lower but stable value under its optimal action.

3. **Number of Iterations**: `4`
   - It took 4 iterations to reach a stable policy, which is reasonable for a small state space like this. Fewer than 10 iterations for policy convergence in small MDPs is typical.


### Summary:
This code demonstrates the **policy iteration** method to solve an MDP by alternating between policy evaluation and improvement. It efficiently converges to an optimal policy through iterative refinement, making it a robust solution for finite MDPs where optimal policies are desirable.